<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Norbert", "transactions": [{"transaction-id": 357, "amount": 1311}, {"transaction-id": 1051, "amount": 1467}, {"transaction-id": 1381, "amount": 1372}, {"transaction-id": 2052, "amount": 1221}, {"transaction-id": 2270, "amount": 1296}, {"transaction-id": 3640, "amount": 1374}, {"transaction-id": 4561, "amount": 1194}, {"transaction-id": 5639, "amount": 1528}, {"transaction-id": 6162, "amount": 1176}, {"transaction-id": 6978, "amount": 889}, {"transaction-id": 8236, "amount": 1126}, {"transaction-id": 9392, "amount": 1351}, {"transaction-id": 9635, "amount": 1060}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Norbert", "transactions": [{"transaction-id": 357, "amount": 1311}, {"transaction-id": 1051, "amount": 1467}, {"transaction-id": 1381, "amount": 1372}, {"transaction-id": 2052, "amount": 1221}, {"transaction-id": 2270, "amount": 1296}, {"transaction-id": 3640, "amount": 1374}, {"transaction-id": 4561, "amount": 1194}, {"transaction-id": 5639, "amount": 1528}, {"transaction-id": 6162, "amount": 1176}, {"transaction-id": 6978, "amount": 889}, {"transaction-id": 8236, "amount": 1126}, {"transaction-id": 9392, "amount": 1351}, {"transaction-id": 9635, "amount": 1060}]}\n',
 '{"id": 1, "name": "Dan", "transactions": [{"transaction-id": 305, "amount": 806}, {"transaction-id": 926, "amount": 819}, {"transaction-id": 1502, "amount": 795}, {"transaction-id": 1911, "amount": 758}, {"transaction-id": 2598, "amount": 874}, {"transaction-id": 2762, "amount": 689}, {"transaction-id": 3271, "amount": 787}, {"transaction-id": 3323, "amount": 770}, {"transaction-id": 4713, "amount"

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Norbert',
  'transactions': [{'transaction-id': 357, 'amount': 1311},
   {'transaction-id': 1051, 'amount': 1467},
   {'transaction-id': 1381, 'amount': 1372},
   {'transaction-id': 2052, 'amount': 1221},
   {'transaction-id': 2270, 'amount': 1296},
   {'transaction-id': 3640, 'amount': 1374},
   {'transaction-id': 4561, 'amount': 1194},
   {'transaction-id': 5639, 'amount': 1528},
   {'transaction-id': 6162, 'amount': 1176},
   {'transaction-id': 6978, 'amount': 889},
   {'transaction-id': 8236, 'amount': 1126},
   {'transaction-id': 9392, 'amount': 1351},
   {'transaction-id': 9635, 'amount': 1060}]},
 {'id': 1,
  'name': 'Dan',
  'transactions': [{'transaction-id': 305, 'amount': 806},
   {'transaction-id': 926, 'amount': 819},
   {'transaction-id': 1502, 'amount': 795},
   {'transaction-id': 1911, 'amount': 758},
   {'transaction-id': 2598, 'amount': 874},
   {'transaction-id': 2762, 'amount': 689},
   {'transaction-id': 3271, 'amount': 787},
   {'transaction-

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 62,
  'name': 'Alice',
  'transactions': [{'transaction-id': 328, 'amount': 300},
   {'transaction-id': 557, 'amount': 296},
   {'transaction-id': 683, 'amount': 328},
   {'transaction-id': 714, 'amount': 306},
   {'transaction-id': 976, 'amount': 314},
   {'transaction-id': 979, 'amount': 337},
   {'transaction-id': 1142, 'amount': 332},
   {'transaction-id': 1241, 'amount': 305},
   {'transaction-id': 1290, 'amount': 316},
   {'transaction-id': 1332, 'amount': 306},
   {'transaction-id': 1351, 'amount': 295},
   {'transaction-id': 1554, 'amount': 325},
   {'transaction-id': 1597, 'amount': 297},
   {'transaction-id': 1874, 'amount': 279},
   {'transaction-id': 1944, 'amount': 322},
   {'transaction-id': 1991, 'amount': 320},
   {'transaction-id': 2392, 'amount': 312},
   {'transaction-id': 3269, 'amount': 308},
   {'transaction-id': 3443, 'amount': 301},
   {'transaction-id': 3576, 'amount': 332},
   {'transaction-id': 3740, 'amount': 322},
   {'transaction-id': 3747, 'amount

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 50},
 {'name': 'Alice', 'count': 11},
 {'name': 'Alice', 'count': 91},
 {'name': 'Alice', 'count': 3},
 {'name': 'Alice', 'count': 91})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(50, 11, 91, 3, 91)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

36.98

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 328, 'amount': 300},
  {'transaction-id': 557, 'amount': 296},
  {'transaction-id': 683, 'amount': 328},
  {'transaction-id': 714, 'amount': 306},
  {'transaction-id': 976, 'amount': 314},
  {'transaction-id': 979, 'amount': 337},
  {'transaction-id': 1142, 'amount': 332},
  {'transaction-id': 1241, 'amount': 305},
  {'transaction-id': 1290, 'amount': 316},
  {'transaction-id': 1332, 'amount': 306},
  {'transaction-id': 1351, 'amount': 295},
  {'transaction-id': 1554, 'amount': 325},
  {'transaction-id': 1597, 'amount': 297},
  {'transaction-id': 1874, 'amount': 279},
  {'transaction-id': 1944, 'amount': 322},
  {'transaction-id': 1991, 'amount': 320},
  {'transaction-id': 2392, 'amount': 312},
  {'transaction-id': 3269, 'amount': 308},
  {'transaction-id': 3443, 'amount': 301},
  {'transaction-id': 3576, 'amount': 332},
  {'transaction-id': 3740, 'amount': 322},
  {'transaction-id': 3747, 'amount': 348},
  {'transaction-id': 3914, 'amount': 305},
  {'transaction-i

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 328, 'amount': 300},
 {'transaction-id': 557, 'amount': 296},
 {'transaction-id': 683, 'amount': 328})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(300, 296, 328)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

3159.9319226608977

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 96), ('Alice', 96), ('Alice', 96), ('Alice', 112), ('Bob', 108), ('Bob', 108), ('Bob', 108), ('Bob', 126), ('Charlie', 108), ('Charlie', 108), ('Charlie', 108), ('Charlie', 126), ('Dan', 108), ('Dan', 108), ('Dan', 108), ('Dan', 126), ('Edith', 51), ('Edith', 53), ('Edith', 54), ('Edith', 62), ('Frank', 84), ('Frank', 84), ('Frank', 84), ('Frank', 98), ('George', 228), ('George', 228), ('George', 228), ('George', 266), ('Hannah', 180), ('Hannah', 180), ('Hannah', 180), ('Hannah', 210), ('Ingrid', 152), ('Ingrid', 153), ('Ingrid', 154), ('Ingrid', 174), ('Jerry', 80), ('Jerry', 81), ('Jerry', 82), ('Jerry', 97), ('Kevin', 72), ('Kevin', 84), ('Kevin', 144), ('Laura', 116), ('Laura', 118), ('Laura', 119), ('Laura', 141), ('Michael', 114), ('Michael', 114), ('Michael', 115), ('Michael', 133), ('Norbert', 128), ('Norbert', 128), ('Norbert', 128), ('Norbert', 151), ('Oliver', 36), ('Oliver', 36), ('Oliver', 36), ('Oliver', 42), ('Patricia', 88), ('Patricia', 89), ('Patricia', 91)

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 400), ('Bob', 450), ('Charlie', 450), ('Dan', 450), ('Edith', 220), ('Frank', 350), ('George', 950), ('Hannah', 750), ('Ingrid', 633), ('Jerry', 340), ('Kevin', 300), ('Laura', 494), ('Michael', 476), ('Norbert', 535), ('Oliver', 150), ('Patricia', 374), ('Quinn', 442), ('Ray', 348), ('Sarah', 526), ('Tim', 300), ('Ursula', 653), ('Victor', 349), ('Wendy', 582), ('Xavier', 543), ('Yvonne', 644), ('Zelda', 500)]
CPU times: user 166 ms, sys: 12 ms, total: 178 ms
Wall time: 418 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Norbert,"[{'transaction-id': 357, 'amount': 1311}, {'tr..."
1,1,Dan,"[{'transaction-id': 305, 'amount': 806}, {'tra..."
2,2,Edith,"[{'transaction-id': 4494, 'amount': 2}, {'tran..."
3,3,Tim,"[{'transaction-id': 338, 'amount': 65}, {'tran..."
4,4,Bob,"[{'transaction-id': 121, 'amount': 996}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 175 ms, sys: 18.1 ms, total: 193 ms
Wall time: 1.23 s


name
Alice      400
Bob        450
Charlie    450
Dan        450
Edith      220
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Norbert', 'amount': 1311, 'transaction-id': 357},
 {'id': 0, 'name': 'Norbert', 'amount': 1467, 'transaction-id': 1051},
 {'id': 0, 'name': 'Norbert', 'amount': 1372, 'transaction-id': 1381})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Norbert,1311,357
1,0,Norbert,1467,1051
2,0,Norbert,1372,1381
3,0,Norbert,1221,2052
4,0,Norbert,1296,2270


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 150 ms, sys: 3.82 ms, total: 154 ms
Wall time: 912 ms


name
Alice       14792
Bob         16959
Charlie     35445
Dan         18334
Edith        8864
Frank       17660
George      34451
Hannah      35430
Ingrid      40468
Jerry       12681
Kevin        6149
Laura       16447
Michael     16981
Norbert     28177
Oliver      13630
Patricia     6999
Quinn       20097
Ray         12901
Sarah       19480
Tim          5245
Ursula      31619
Victor      12762
Wendy       20500
Xavier      12398
Yvonne      20152
Zelda       21379
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()